In [44]:
import os.path
import numpy as np
import copy
import networkx as nx
import os
import pickle
import time

In [45]:
arquivo = os.path.join('projeto.csv')
dataRDD = (sc.textFile(arquivo, 8))
RDD1 = dataRDD.map(lambda l: l.split(',')).map(lambda l: {'label':float(l[0]),'features':[float(l[1]),float(l[2]),l[3]]})
RDD = RDD1.filter(lambda v: v['label']<=1.61) #removing outliers
weights = [.999,.001]
seed = 42
lpRDD, lpTest = RDD.randomSplit(weights, seed)

In [46]:
def contador(lp, feats, splits):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    r = []
    for feat in feats:
        splitss = splits[feat]
        for split in splitss:
            if feat==3:
                r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
            elif feat==0 or feat==1:
                if lp['features'][feat] >= split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
            elif feat==2:                
                if lp['features'][feat] in split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
    return r

def soma(t1,t2):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return  ((t1[0][0]+t2[0][0],t1[0][1]+t2[0][1]), 
             (t1[1][0]+t2[1][0],t1[1][1]+t2[1][1]),
             (t1[2][0]+t2[2][0],t1[2][1]+t2[2][1]))

In [47]:
def calc_stats(t):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    if t[1][2][0] > 0:
        mmenor = t[1][0][0] / t[1][2][0]
        sdmenor = (t[1][1][0] - (t[1][0][0]**2 / t[1][2][0]))/(t[1][2][0])
        cvmenor = np.sqrt(sdmenor)/mmenor*100
        menor = (sdmenor, cvmenor,t[1][2][0])
    else:
        menor = (0, 0, 0)
    
    if t[1][2][1] > 0:    
        mmaior = t[1][0][1] / t[1][2][1]
        sdmaior = (t[1][1][1] - (t[1][0][1]**2 / t[1][2][1]))/(t[1][2][1])
        cvmaior = np.sqrt(sdmaior)/mmaior*100
        maior = (sdmaior, cvmaior,t[1][2][1])
    else:
        maior = (0, 0, 0)
    return (t[0], menor, maior)

def informationGain(t,c):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return ((t[0], c[2][0] - (t[1][2]/c[2][2])*t[1][0] - (t[2][2]/c[2][2])*t[2][0], (t[1][1], t[2][1]))) 

In [48]:
def preparaMMU(t):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """    
    return ((t['features'][0],t['features'][0]),(t['features'][1],t['features'][1]), set((t['features'][2],)))
    
def minmax_unique(t1,t2):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return ((min(t1[0][0],t2[0][0]),max(t1[0][1],t2[0][1])),
            (min(t1[1][0],t2[1][0]),max(t1[1][1],t2[1][1])),
             t1[2].union(t2[2]))

In [49]:
def predict(arvore, lp, i=0):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    if type(arvore[i]) != tuple:
        if arvore[i]!=0:
            return (lp['label'], arvore[i])
        else:
            return (lp['label'], lp['label'])
    
    node = arvore[i]

    if node[0]<2:
        if lp['features'][node[0]]<node[1]:
            return predict(arvore, lp, 2*i+1)
        else:
            return predict(arvore, lp, 2*i+2)
    elif node[0]==2:
        if lp['features'][node[0]] in node[1]:
            return predict(arvore, lp, 2*i+1)
        else:
            return predict(arvore, lp, 2*i+2)
        
    

In [59]:
def makeSplits(lpRDD):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    (min0,max0),(min1,max1),periods =lpRDD.map(preparaMMU).reduce(minmax_unique)
    
    periods = list(periods)

    
    distsplits = [i*(max0-min0)/5. for i in range(1,5)]
    timesplits = [i*(max1-min1)/5. for i in range(1,5)]    
    perisplits = [tuple([periods[j] for j in range(i+1)]) for i in range(len(periods)-1)]

    feats = [0,1,2,3]
    splits = [distsplits,timesplits,perisplits,['full']] 
    return (feats,splits)

In [60]:
def infoGainSplit(lpRDD,feats,splits):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    results = lpRDD.flatMap(lambda lp: contador(lp,feats,splits)).reduceByKey(soma)    
    statistics = results.map(lambda c: calc_stats(c))    
    classe = statistics.filter(lambda x: x[0][0]==3).collect()[0]
    es = statistics.filter(lambda x: x[0][0]<3).map(lambda x: informationGain(x,classe)).collect()          
    e = max(es, key = lambda t: t[1]) #atributo / split escolhido   
    return e

In [61]:
def splitData(lpRDD, e):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    if e[0][0]<2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]<e[0][1])
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]>e[0][1])
    elif e[0][0]==2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] in e[0][1])
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] not in e[0][1])
    return (menorRDD,maiorRDD)

In [62]:
def savetree(arvore, filename):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    with open(filename, 'wb') as handle:
        pickle.dump(arvore, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [63]:
def readtree(filename):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    with open(filename, 'rb') as handle:
        arvore = pickle.load(handle)
    return arvore

In [64]:
def calcError(lpTest, arvore):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    sqr = lpTest.map(lambda x: predict(arvore, x)).map(lambda x: (x[0]-x[1])**2).sum()
    return sqr

In [65]:
def treeimage(arvore):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    G = nx.DiGraph()
    for v,l in arvore.items():
        if type(l)==tuple:
            if type(l[1])!=tuple:
                G.add_node(v,label=(l[0],'{0:.2f}'.format(l[1])))
            else:
                G.add_node(v,label=l)
            
        else:
            G.add_node(v,label='{0:.2f}'.format(l))

    for v in G.nodes():
    
        if 2*v+1 in G:
            G.add_edge(v,2*v+1)
        if 2*v+2 in G:
            G.add_edge(v,2*v+2)

    os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
    p=nx.drawing.nx_pydot.to_pydot(G)
    p.write_png('arvore4.png')

In [69]:
def buildTree(lpRDD, maxDepth=1, currDepth=0, st=None, arvore={}, pos=0):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    if st!=None:
        if st[0]<10 or maxDepth<currDepth or st[1]<0:
            media = lpRDD.map(lambda x: x['label']).mean()
            arvore[pos] = media
            return arvore 
    

    feats,splits = makeSplits(lpRDD)
    
    e = infoGainSplit(lpRDD,feats,splits)
    
    menorRDD,maiorRDD = splitData(lpRDD, e)
    
    arvore[pos] = e[0]
    
    amenor = buildTree(menorRDD, maxDepth, currDepth+1,(e[2][0], e[1]),arvore,pos=(2*pos+1))
    amaior = buildTree(maiorRDD, maxDepth, currDepth+1,(e[2][1], e[1]),arvore,pos=(2*pos+2))
    return arvore

In [70]:
ti=time.time()
arvore = buildTree(lpRDD,maxDepth=1)
tf = time.time()
print('Tempo total de construção paralelizado (local[4]): %.2f segundos'%(tf-ti))

Tempo total de construção paralelizado (local[4]): 1379.77 segundos


In [72]:
print(calcError(lpTest, arvore))

103.74703903274495
